In [4]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
def build_data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):

    transform = transforms.Compose([transforms.ToTensor()])
    
    train_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    valid_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)
  
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_loader, valid_loader, test_loader)

In [3]:
def train(model, data_loader, valid_loader, num_epochs, criterion, optimizer, device):
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for step, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
        
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print ('Step [{}/{}], Loss: {:.4f}'.format(step+1, total_steps, loss.item()))
               
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            

In [4]:
def validate(model, valid_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
    print('Accuracy of the network on the {} validation images: {:.2f} %'.format(5000, 100 * correct / total)) 

In [5]:
def test(model, test_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))  

In [6]:
def count_parameters(model):
    parameters = list(model.parameters())
    total_parms = sum([np.prod(p.size()) for p in parameters if p.requires_grad])
    return total_parms


In [80]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.fc1 = nn.Linear(64*7*7, 512)
        
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [81]:
# General parameters
data_dir = '/tmp'
device = 'cpu'
num_classes = 10

# Hyperparameters
max_lr = 0.00001
weight_decay = 0.005
batch_size = 64
learning_rate = 0.0001
num_epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

# FashionMNIST dataset 
train_loader, valid_loader, test_loader = build_data_loader(data_dir=data_dir, batch_size=batch_size)

# Model definition
model = CNN()

# Optimizer
optimizer = optimizer(model.parameters(), max_lr, weight_decay=weight_decay)

In [82]:
print(count_parameters(model))

1758858


In [83]:
%%time
# Train the model
train(model, train_loader, valid_loader, num_epochs, criterion, optimizer, device)
test(model, test_loader, device)

Epoch [1/5], Loss: 0.7138
Epoch [2/5], Loss: 0.5603
Epoch [3/5], Loss: 0.4461
Epoch [4/5], Loss: 0.6135
Epoch [5/5], Loss: 0.4535
Accuracy of the network on the 10000 test images: 80.25 %
CPU times: user 12min 42s, sys: 41 s, total: 13min 23s
Wall time: 51.9 s


In [35]:
schedule = torch.profiler.schedule(wait=5, warmup=5, active=3)

#prof = profile(schedule=schedule, activities=[ProfilerActivity.CPU], with_stack=True, record_shapes=True, on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/cnn/'))
prof = profile(activities=[ProfilerActivity.CPU])
        
input_sample, _ = next(iter(train_loader))

prof.start()
model(input_sample)
prof.stop()

#prof.export_chrome_trace("./cnn_trace.json")
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=100))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::mkldnn_convolution        44.01%       1.264ms        44.60%       1.281ms     640.500us             2  
    aten::max_pool2d_with_indices        30.01%     862.000us        30.01%     862.000us     431.000us             2  
                      aten::addmm        13.68%     393.000us        14.52%     417.000us     208.500us             2  
                  aten::clamp_min         6.96%     200.000us         6.96%     200.000us     100.000us             2  
                aten::convolution         1.18%      34.000us        46.27%       1.329ms     664.500us             2  
                      aten::copy_       

STAGE:2023-05-02 16:43:31 105274:105274 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-05-02 16:43:31 105274:105274 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-05-02 16:43:31 105274:105274 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [27]:
input_names = ['input']
output_names = ['output']
torch.onnx.export(model, input_sample, "cnn.onnx", input_names=input_names, output_names=output_names, export_params=True)

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

